In [1]:
import torch
import torch.nn as nn
from torchvision import datasets ,models,transforms
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.nn import Linear, ReLU, CrossEntropyLoss, Conv2d, MaxPool2d, Module
from torch.optim import Adam
import pandas as pd
import os
from os import listdir
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from dataloader_teeth import *
from torch.utils.data import DataLoader
from torchsummary import summary
import torch.optim as optim
from torch.nn import *
from EarlyStopping import EarlyStopping
from models import *
#from pytorchtools import EarlyStopping

In [12]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(y_true, y_pred, title):
    
    confusion = confusion_matrix(y_true, y_pred)
    classes = [i for i in range(1,33)]
    confusion = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis] #normalize
    
    
    fig, ax = plt.subplots(figsize=(15,15))
    im = ax.imshow(confusion, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)

    ax.set(xticks=np.arange(confusion.shape[1]),
           yticks=np.arange(confusion.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    
    fmt = '.2f'
    thresh = confusion.max() / 2.
    for i in range(confusion.shape[0]):
        for j in range(confusion.shape[1]):
            ax.text(j, i, format(confusion[i, j], fmt),
                    ha="center", va="center",
                    color="white" if confusion[i, j] > thresh else "black")
    fig.tight_layout()
    plt.savefig(title + ".png")
    plt.clf()
    plt.cla()
    plt.close()
    
    return ax

def revised_by_table(radiograph, uni):
    if (uni==4) or (uni==5):
        radiograph.iloc[0] = [4,13,True]
        radiograph.iloc[1] = [5,12,True]
    elif (uni==8) or (uni==9):
        radiograph.iloc[0] = [8,9,True]
        radiograph.iloc[1] = [9,8,True]
    elif (uni==12) or (uni==13):
        radiograph.iloc[0] = [12,5,True]
        radiograph.iloc[1] = [13,4,True]
    elif (uni==20) or (uni==21):
        radiograph.iloc[0] = [21,28,True]
        radiograph.iloc[1] = [20,29,True]
    elif (uni==28) or (uni==29):
        radiograph.iloc[0] = [29,20,True]
        radiograph.iloc[1] = [28,21,True]
    return radiograph
    
def only_onefault(radiograph):
    if len(radiograph[radiograph['check']==True]['p'].unique()) == len(radiograph[radiograph['check']==True])-1:
        mean = radiograph[radiograph['check']==True]['p'].mean()

        x = radiograph[radiograph['check']==True]['p'].tolist()
        print(x)
        if mean <= 17:
            v_label = 17
            sort_ok = True
            for i in range(len(x)-1):
                if x[i]>x[i+1]:
                    sort_ok = False
            if sort_ok == True:
                count_bin = np.zeros(32,dtype=int)
                for radio_idx, radio_row in radiograph[radiograph['check']==True].iterrows():
                    count_bin[radio_row['p']-1] += 1

                ok_list = []
                once = False
                for i in range(len(x)):
                    if count_bin[x[i]-1] == 2:
                        if once == True:
                            continue
                        once = True
                        if x[i] <= mean:
                            if count_bin[x[i]] == 0:
                                ok_list.append(x[i])
                                ok_list.append(x[i]+1)
                            elif count_bin[x[i]-2] == 0:
                                ok_list.append(x[i]-1)
                                ok_list.append(x[i])
                            else:
                                ok_list.append(x[i])
                                ok_list.append(x[i])
                        else:
                            if count_bin[x[i]-2] == 0:
                                ok_list.append(x[i]-1)
                                ok_list.append(x[i])
                            elif count_bin[x[i]] == 0:
                                ok_list.append(x[i])
                                ok_list.append(x[i]+1)
                            else:
                                ok_list.append(x[i])
                                ok_list.append(x[i])
                    else:
                        ok_list.append(x[i])

                v_idx = 0
                for radio_idx, radio_row in radiograph[radiograph['check']==True].iterrows():
                    radiograph.loc[radio_idx] = [ok_list[v_idx],v_label-ok_list[v_idx],True] 
                    v_idx += 1
        else:
            v_label = 49
            sort_ok = True
            for i in range(len(x)-1):
                if x[i]<x[i+1]:
                    sort_ok = False
            if sort_ok == True:
                count_bin = np.zeros(32,dtype=int)
                for radio_idx, radio_row in radiograph[radiograph['check']==True].iterrows():
                    count_bin[radio_row['p']-1] += 1

                ok_list = []
                once = False
                for i in range(len(x)):
                    if count_bin[x[i]-1] == 2:
                        if once == True:
                            continue
                        once = True
                        if x[i] <= mean:
                            if count_bin[x[i]] == 0:
                                ok_list.append(x[i]+1)
                                ok_list.append(x[i])
                            elif count_bin[x[i]-2] == 0:
                                ok_list.append(x[i])
                                ok_list.append(x[i]-1)
                            else:
                                ok_list.append(x[i])
                                ok_list.append(x[i])
                        else:
                            if count_bin[x[i]-2] == 0:
                                ok_list.append(x[i])
                                ok_list.append(x[i]-1)
                            elif count_bin[x[i]] == 0:
                                ok_list.append(x[i]+1)
                                ok_list.append(x[i])
                            else:
                                ok_list.append(x[i])
                                ok_list.append(x[i])
                    else:
                        ok_list.append(x[i])

                v_idx = 0
                for radio_idx, radio_row in radiograph[radiograph['check']==True].iterrows():
                    radiograph.loc[radio_idx] = [ok_list[v_idx],v_label-ok_list[v_idx],True] 
                    v_idx += 1
    return radiograph


In [24]:
def count_point_table(radiograph):
    point_table = np.zeros(len(radiograph),dtype=int)
    print('len(point_table) =',len(radiograph))
    if radiograph['p'].mean() < 17:
        v_value = 17
        for radio_idx, radio_row in radiograph.iterrows():#加起來17
            if radio_row['p']+radio_row['fp'] == 17:
                point_table[radio_idx] += 1

        for i in range(len(radiograph)-1): #正向分數
            for j in range(i+1,len(radiograph)):
                if radiograph.iloc[j]['p'] - radiograph.iloc[i]['p'] == j - i:
                    point_table[i] += 1
                    point_table[j] += 1

        for i in range(len(radiograph)-1): #逆向分數
            for j in range(i+1,len(radiograph)):
                if radiograph.iloc[j]['fp'] - radiograph.iloc[i]['fp'] == i - j:
                    point_table[i] += 1
                    point_table[j] += 1
    else:
        v_value = 49
        for radio_idx, radio_row in radiograph.iterrows():#加起來49
            if radio_row['p']+radio_row['fp'] == 49:
                point_table[radio_idx] += 1

        for i in range(len(radiograph)-1):
            for j in range(i+1,len(radiograph)):
                if radiograph.iloc[j]['p'] - radiograph.iloc[i]['p'] == i - j:
                    point_table[i] += 1
                    point_table[j] += 1

        for i in range(len(radiograph)-1):
            for j in range(i+1,len(radiograph)):
                if radiograph.iloc[j]['fp'] - radiograph.iloc[i]['fp'] == j - i:
                    point_table[i] += 1
                    point_table[j] += 1

    print(point_table)
    return (point_table, v_value)

def revised(radiograph, point_table, v_value):
    max_num = point_table.max()
    if len(point_table) == 1:
        return radiograph
    
    base = 0
    for i in range(len(point_table)):
        if point_table[i]==max_num:
            base = i
            break
    
    if v_value == 17:
        print('')
        for radio_idx in range(len(point_table)):
            if point_table[radio_idx]!=max_num:
                gap = radio_idx - base
                revise_value = radiograph.iloc[base]['p']+gap
                radiograph.iloc[radio_idx] = [revise_value,v_value-revise_value,False]
            else:
                if radiograph.iloc[radio_idx]['check'] == False:
                    gap = radio_idx - base
                    revise_value = radiograph.iloc[base]['p']+gap
                    radiograph.iloc[radio_idx] = [revise_value,v_value-revise_value,False]
                else:
                    base = radio_idx
    else:
        for radio_idx in range(len(point_table)):
            if point_table[radio_idx]!=max_num:
                gap = radio_idx - base
                revise_value = radiograph.iloc[base]['p']-gap
                radiograph.iloc[radio_idx] = [revise_value,v_value-revise_value,False] 
            else:
                if radiograph.iloc[radio_idx]['check'] == False:
                    gap = radio_idx - base
                    revise_value = radiograph.iloc[base]['p']-gap
                    radiograph.iloc[radio_idx] = [revise_value,v_value-revise_value,False]
                else:
                    base = radio_idx 
            
    return radiograph

In [25]:
class DeepConvNet(torch.nn.Module):
    def __init__(self, act_fn='ReLU'):
        super().__init__()
        
        dropout = 0.2
        self.conv0 = Conv2d(1, 32, kernel_size=(3, 3))
        self.conv1 = Conv2d(32, 32, kernel_size=(3, 3))
        self.layer1 = Sequential(
            BatchNorm2d(32),
            self.activationFunc(act_fn),
            MaxPool2d(kernel_size=(2, 2)),
            #Dropout2d(dropout)
        )

        self.conv2 = Conv2d(32, 64, kernel_size=(3, 3))
        self.layer2 = Sequential(
            BatchNorm2d(64),
            self.activationFunc(act_fn),
            MaxPool2d(kernel_size=(2, 2)),
            #Dropout(dropout)
        )
        
        self.conv3 = Conv2d(64, 128, kernel_size=(3, 3))
        self.layer3 = Sequential(
            BatchNorm2d(128),
            self.activationFunc(act_fn),
            MaxPool2d(kernel_size=(2, 2)),
            #Dropout(dropout)
        )
        
        self.conv4 = Conv2d(128, 128, kernel_size=(3, 3))
        self.layer4 = Sequential(
            BatchNorm2d(128),
            self.activationFunc(act_fn),
        )
        
        self.maxpool = Sequential(
            MaxPool2d(kernel_size=(2, 2)),
            #Dropout(dropout)
        )  

        self.fc1 = Sequential(
            Linear(in_features=8960,out_features=128),
            self.activationFunc(act_fn),
            Dropout(0.4)
        )

        self.fc2 = Sequential(
            Linear(in_features=128,out_features=64),
            self.activationFunc(act_fn),
            Dropout(0.2)
        )

        self.classify = Sequential(
            Linear(in_features=64,out_features=32),
        )


    def activationFunc(self, act_fn):
        if act_fn == 'ReLU':
            return ReLU()
        elif act_fn == 'LReLU':
            return LeakyReLU()
        else:
            return ELU()
        
    def activations_hook(self, grad):
        self.gradients = grad
        
    def forward(self, x):
        x = self.conv0(x)
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.conv2(x)
        x = self.layer2(x)
        x = self.conv3(x)
        x = self.layer3(x)
        x = self.conv4(x)
        x = self.layer4(x)
        
        h = x.register_hook(self.activations_hook)
        
        
        x = self.maxpool(x)
        #x = x.view(-1, 21504) # 當bug為 tensor_a(4).....tensor_b(8) -> //2 ->476800//2 (150,200)
        x = x.view(-1, 8960)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.classify(x)
        
        #print('final size =',x.size())
        return x
    
    # method for the gradient extraction
    def get_activations_gradient(self):
        return self.gradients
    
    # method for the activation exctraction
    def get_activations(self, x):
        x = self.conv0(x)
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.conv2(x)
        x = self.layer2(x)
        x = self.conv3(x)
        x = self.layer3(x)
        x = self.conv4(x)
        x = self.layer4(x)
        return x

print(DeepConvNet)

<class '__main__.DeepConvNet'>


In [26]:
%%time
test_dataset = GetDataset('','test',Istestori=True)
testloader = DataLoader(test_dataset, batch_size=1,shuffle=False)

data length = 9552
class 1:	length :6
class 2:	length :69
class 3:	length :60
class 4:	length :81
class 5:	length :73
class 6:	length :79
class 7:	length :73
class 8:	length :74
class 9:	length :74
class 10:	length :73
class 11:	length :79
class 12:	length :73
class 13:	length :81
class 14:	length :60
class 15:	length :69
class 16:	length :6
class 17:	length :9
class 18:	length :56
class 19:	length :52
class 20:	length :67
class 21:	length :90
class 22:	length :66
class 23:	length :87
class 24:	length :58
class 25:	length :58
class 26:	length :87
class 27:	length :66
class 28:	length :90
class 29:	length :67
class 30:	length :52
class 31:	length :56
class 32:	length :9
> Found 2000 images...
Wall time: 625 ms


In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Create test consecutive data in dental radiograph

In [28]:
# count = 0
# df = pd.DataFrame(columns=['folder','image','label','predict','total_path'])
# for idx, data in tqdm(enumerate(testloader)):
#     img, label, img_path = data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.int64), data[2]
#     if img_path[0][-5] == 'F':
#         continue
#     x, y = data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.int64)
#     output = model(x)
#     max_value, predicted = torch.max(output.data, 1)
    
#     if len(img_path[0].split('_'))==14:
#         foloder = img_path[0].split('_')[-7]
#     if len(img_path[0].split('_'))==15:
#         foloder = img_path[0].split('_')[-8] + '_' + img_path[0].split('_')[-7]
        
#     image = img_path[0].split('_')[-3]
#     label = label.cpu().numpy()[0]+1
#     predict = predicted.cpu().numpy()[0]+1
#     df.loc[count] = [foloder, image, label, predict, img_path[0]]
#     count += 1
#     #print('img_path =',img_path[0])

# df = df.set_index(keys = ['folder','image'])
# df.sort_values(['folder', 'image','label'],inplace=True)

# for idx in df.index.unique():
#     if df.loc[idx]['label'][0] > 16:
#         df.loc[idx] = df.loc[idx].sort_values(['folder', 'image','label'],ascending=[True, True, False])
# #df.to_csv('0824_only_nonflip.csv')
# df.to_csv('0831_only_nonflip.csv')
# df

In [29]:
# len_dataframe = pd.DataFrame(columns=['folder','image','length'])
# for idx, folder_image in enumerate(df.index.unique()):
#     len_dataframe.loc[idx] = [folder_image[0],folder_image[1],len(df.loc[folder_image])]
# len_dataframe = len_dataframe.set_index(keys = ['folder','image'])
# len_dataframe.to_csv('len_dataframe.csv')
# len_dataframe

# Algorithm

In [30]:
len_dataframe = pd.read_csv('len_dataframe_0824.csv')
len_dataframe = len_dataframe.set_index(keys = ['folder','image'])
df = pd.read_csv('0824_only_nonflip.csv')
df = df.set_index(keys = ['folder','image'])
df

label  predict  \
folder            image                    
585387_2 091119 x 4C085      30       30   
                  4C085      29       29   
                  4C085      28       28   
                  4C086      10       10   
                  4C086      11       11   
...                         ...      ...   
684988 92118 x    16DEE8     21       21   
                  16DEE9      2        2   
                  16DEE9      5        4   
                  16DEEA     28       28   
                  16DEEA     27       27   

                                                                 total_path  
folder            image                                                      
585387_2 091119 x 4C085   Dental_Data/PBL/PBL_addname/10_clahe_20200810_...  
                  4C085   Dental_Data/PBL/PBL_addname/10_clahe_20200810_...  
                  4C085   Dental_Data/PBL/PBL_addname/10_clahe_20200810_...  
                  4C086   Dental_Data/PBL/PBL_addname/10_clahe_20200810_...  
                  4C086   Dental_Data/PBL/PBL_addname/10_clahe_20200810_...  
...                                                                     ...  
684988 92118 x    16DEE8  Dental_Data/PBL/PBL_addname/10_clahe_20200810_...  
                  16DEE9  Dental_Data/PBL/PBL_addname/10_clahe_20200810_...  
                  16DEE9  Dental_Data/PBL/PBL_addname/10_clahe_20200810_...  
                  16DEEA  Dental_Data/PBL/PBL_addname/10_clahe_20200810_...  
                  16DEEA  Dental_Data/PBL/PBL_addname/10_clahe_20200810_...  

[1000 rows x 3 columns]

In [31]:
new_size = (200,150)
trans_method = [
    transforms.Resize(new_size),
    transforms.Grayscale(),
    transforms.ToTensor()
]
trans_flipback = [
    transforms.Resize(new_size),
    transforms.RandomVerticalFlip(p=1),
    transforms.Grayscale(),
    transforms.ToTensor()
]
trans = transforms.Compose(trans_method)
trans_back = transforms.Compose(trans_flipback)

In [32]:
model = torch.load('./model/best_10degree.pkl')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
%%time
count = 0
cur_index = len_dataframe.iloc[count].values[0]

out_df = pd.DataFrame(columns=['folder','image','label','flip_label','p','fp','check'])
out_df_idx = 0

radiograph = pd.DataFrame(columns=['p','fp','check'])
radiograph_idx = 0
ori_df = pd.DataFrame(columns=['folder','image','ori_p','ori_fp'])

total_point_table = []

for idx, row in enumerate(df.iterrows()):
    if idx >= cur_index: # next radiograph
        
        radiograph_idx = 0
        radiograph = pd.DataFrame(columns=['p','fp','check'])
        
        count += 1
        cur_index += len_dataframe.iloc[count].values[0]
#########################input and prediction#########################    
    img_path = row[1]['total_path']
    img = PIL.Image.open(img_path).convert('RGB')
    flip_img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
    
    label = row[1]['label']
    if label < 17:
        flabel = 17 - label
        img = trans_back(img)
        flip_img = trans_back(flip_img)
    else:
        flabel = 49 - label
        img = trans(img)
        flip_img = trans(flip_img)
    
    img = img.to(device)
    img.unsqueeze_(0)
    flip_img = flip_img.to(device)
    flip_img.unsqueeze_(0)
    
    output = model(img)
    max_value, predicted = torch.max(output.data, 1)
    output = model(flip_img)
    max_value, flip_predicted = torch.max(output.data, 1)
    p = predicted.cpu().numpy()[0]+1
    fp = flip_predicted.cpu().numpy()[0]+1
    #print('truth = {},predicted = {}, flip_truth = {}, flip_predicted = {}'.format(label,p,flabel,fp))
    ori_df.loc[idx] = [row[0][0],row[0][1],p,fp]
#########################input and prediction#########################
    if (p+fp==17) | (p+fp==49):
        check = True
    else:
        check = False
    
    radiograph.loc[radiograph_idx] = [p, fp, check]
    radiograph_idx += 1
    if idx == cur_index-1: # do rule operation
        #print('size =',len_dataframe.iloc[count].values[0])
        
        print(row[0])
        
        #全錯先不處理
        if len(radiograph[radiograph['check']==False]) == len(radiograph):
            for i in range(len(radiograph)):
                
                label = df.loc[row[0],['label']].iloc[i][0]
                if label < 17:
                    flabel = 17 - label
                else:
                    flabel = 49 - label

                out_df.loc[out_df_idx] = [row[0][0],row[0][1],label,flabel,radiograph.iloc[i]['p'],radiograph.iloc[i]['fp'],radiograph.iloc[i]['check']]
                out_df_idx += 1
                
                total_point_table+=[999]
                
            continue
        
        point_table, v_value = count_point_table(radiograph)
        
        radiograph = revised(radiograph, point_table, v_value)
        
        if (len(radiograph[radiograph['check']==True]) == len(radiograph)) and (len(radiograph)==2):
            if len(radiograph['p'].unique()) == 1:
                print('uniqyeeeeeeeeeeeeeeeeee')
                uni = radiograph['p'].unique()[0]
                radiograph = revised_by_table(radiograph,uni)
        
        for i in range(len(radiograph)):
            label = df.loc[row[0],['label']].iloc[i][0]
            if label < 17:
                flabel = 17 - label
            else:
                flabel = 49 - label
            out_df.loc[out_df_idx] = [row[0][0],row[0][1],label,flabel,radiograph.iloc[i]['p'],radiograph.iloc[i]['fp'],radiograph.iloc[i]['check']]
            out_df_idx += 1
        
####################################################################
        total_point_table += point_table.tolist()
        radiograph['ori_p'] = df.loc[row[0],['label']].values
        radiograph['ori_fp'] = v_value - df.loc[row[0],['label']].values
        print(pd.DataFrame(radiograph))
        print()
out_df['point'] = total_point_table

('585387_2 091119 x', '4C085')
len(point_table) = 3
[5 5 5]
    p  fp check  ori_p  ori_fp
0  30  19  True     30      19
1  29  20  True     29      20
2  28  21  True     28      21

('585387_2 091119 x', '4C086')
len(point_table) = 3
[5 5 5]

    p fp check  ori_p  ori_fp
0  10  7  True     10       7
1  11  6  True     11       6
2  12  5  True     12       5

('585387_2 091119 x', '4C087')
len(point_table) = 2
[3 3]

    p fp check  ori_p  ori_fp
0  12  5  True     12       5
1  13  4  True     13       4

('585387_2 091119 x', '4C08A')
len(point_table) = 2
[3 3]

    p fp check  ori_p  ori_fp
0  12  5  True     12       5
1  13  4  True     13       4

('585387_2 091119 x', '4C08B')
len(point_table) = 1
[1]
   p  fp check  ori_p  ori_fp
0  6  11  True      6      11

('585387_2 091119 x', '4C090')
len(point_table) = 2
[3 3]
    p  fp check  ori_p  ori_fp
0  19  30  True     19      30
1  18  31  True     18      31

('585387_2 091119 x', '4C091')
len(point_table) = 4
[7 7 7 7]
  

('600872 041519 x', '15BF43')
len(point_table) = 2
[3 3]
    p  fp check  ori_p  ori_fp
0  21  28  True     21      28
1  20  29  True     20      29

('600872 041519 x', '15BF44')
len(point_table) = 1
[1]
   p  fp check  ori_p  ori_fp
0  4  13  True      4      13

('600872 041519 x', '15BF46')
len(point_table) = 2
[1 0]

    p fp  check  ori_p  ori_fp
0  13  4   True     12       5
1  14  3  False     13       4

('600872 041519 x', '15BF48')
len(point_table) = 3
[1 3 3]
    p  fp  check  ori_p  ori_fp
0  28  21  False     28      21
1  27  22   True     27      22
2  26  23   True     26      23

('600872 041519 x', '15BF49')
len(point_table) = 3
[5 5 5]

   p  fp check  ori_p  ori_fp
0  7  10  True      7      10
1  8   9  True      8       9
2  9   8  True      9       8

('600872 041519 x', '15BF4B')
len(point_table) = 1
[1]
    p  fp check  ori_p  ori_fp
0  30  19  True     31      18

('600872 041519 x', '15BF4E')
len(point_table) = 3
[5 5 5]
    p  fp check  ori_p  ori_fp
0  2

('627007 072319 x', '6A815')
len(point_table) = 3
[5 5 5]
    p  fp check  ori_p  ori_fp
0  30  19  True     30      19
1  29  20  True     29      20
2  28  21  True     28      21

('627007 072319 x', '6A816')
len(point_table) = 3
[1 1 1]

    p  fp check  ori_p  ori_fp
0   7  10  True      8       9
1   9   8  True      9       8
2  11   6  True     10       7

('627007 072319 x', '6A817')
len(point_table) = 2
[1 0]

   p  fp  check  ori_p  ori_fp
0  4  13   True      4      13
1  5  12  False      5      12

('627007 072319 x', '6A81C')
len(point_table) = 4
[7 7 7 7]
    p  fp check  ori_p  ori_fp
0  31  18  True     31      18
1  30  19  True     30      19
2  29  20  True     29      20
3  28  21  True     28      21

('627007 072319 x', '6A81D')
len(point_table) = 1
[1]
   p  fp check  ori_p  ori_fp
0  6  11  True      6      11

('627007 072319 x', '6A81E')
len(point_table) = 4
[3 1 1 3]

    p fp  check  ori_p  ori_fp
0  11  6   True     11       6
1  12  5  False     12      

[3 3]

   p  fp check  ori_p  ori_fp
0  2  15  True      2      15
1  3  14  True      3      14

('652530 030818 x', '16FA71')
len(point_table) = 1
[1]
    p fp check  ori_p  ori_fp
0  10  7  True     10       7

('674179 101218 x', '16D10F')
len(point_table) = 2
[1 1]
uniqyeeeeeeeeeeeeeeeeee
    p  fp check  ori_p  ori_fp
0  21  28  True     21      28
1  20  29  True     20      29

('674179 101218 x', '16D110')
len(point_table) = 1
[1]
   p fp check  ori_p  ori_fp
0  8  9  True      8       9

('674179 101218 x', '16D111')
len(point_table) = 2
[1 1]

   p  fp check  ori_p  ori_fp
0  6  11  True      6      11
1  8   9  True      7      10

('674179 101218 x', '16D112')
len(point_table) = 3
[5 5 5]
    p  fp check  ori_p  ori_fp
0  25  24  True     25      24
1  24  25  True     24      25
2  23  26  True     23      26

('674179 101218 x', '16D115')
len(point_table) = 2
[0 1]

   p  fp  check  ori_p  ori_fp
0  4  13  False      4      13
1  5  12   True      5      12

('674179 101

('676633_2 082918 x', '16D421')
len(point_table) = 4
[0 0 3 3]

    p fp  check  ori_p  ori_fp
0  12  5  False     12       5
1  13  4  False     13       4
2  14  3   True     14       3
3  15  2   True     15       2

('676633_2 082918 x', '16D422')
len(point_table) = 2
[1 1]

   p  fp check  ori_p  ori_fp
0  2  15  True      2      15
1  4  13  True      4      13

('676633_2 082918 x', '16D424')
len(point_table) = 3
[0 0 1]

   p  fp  check  ori_p  ori_fp
0  5  12  False     11       6
1  6  11  False     12       5
2  7  10   True     13       4

('676633_2 082918 x', '16D426')
len(point_table) = 1
[1]
   p  fp check  ori_p  ori_fp
0  7  10  True      7      10

('676633_2 082918 x', '16D428')
len(point_table) = 3
[3 1 1]
    p  fp  check  ori_p  ori_fp
0  29  20   True     29      20
1  28  21  False     28      21
2  27  22  False     27      22

('676633_2 082918 x', '16D42A')
len(point_table) = 3
[4 2 4]
    p  fp  check  ori_p  ori_fp
0  23  26   True     23      26
1  22  27

('679674 102618 x', '16CDB3')
len(point_table) = 2
[3 3]

   p fp check  ori_p  ori_fp
0  8  9  True      8       9
1  9  8  True      9       8

('679674 102618 x', '16CDB4')
len(point_table) = 3
[4 2 4]

    p fp  check  ori_p  ori_fp
0  11  6   True     11       6
1  12  5  False     12       5
2  13  4   True     13       4

('679674 102618 x', '16CDB5')
len(point_table) = 2
[2 1]

    p fp  check  ori_p  ori_fp
0  11  6   True     11       6
1  12  5  False     12       5

('679674 102618 x', '16CDB8')
len(point_table) = 3
[3 0 3]
    p  fp  check  ori_p  ori_fp
0  26  23   True     26      23
1  25  24  False     25      24
2  24  25   True     24      25

('679674 102618 x', '16CDB9')
len(point_table) = 3
[5 5 5]
    p  fp check  ori_p  ori_fp
0  28  21  True     28      21
1  27  22  True     27      22
2  26  23  True     26      23

('679674 102618 x', '16CDBA')
len(point_table) = 2
[3 3]

    p fp check  ori_p  ori_fp
0   9  8  True      9       8
1  10  7  True     10      

('681584 061518 x', '16F1F0')
len(point_table) = 3
[4 4 2]
    p  fp  check  ori_p  ori_fp
0  28  21   True     28      21
1  27  22   True     27      22
2  26  23  False     26      23

('681584 061518 x', '16F1F1')
len(point_table) = 1
[1]
   p  fp check  ori_p  ori_fp
0  7  10  True      7      10

('681584 061518 x', '16F1F2')
len(point_table) = 1
[1]
    p  fp check  ori_p  ori_fp
0  31  18  True     31      18

('681916 083118 x', '16E64D')
len(point_table) = 3
[4 2 4]
    p  fp  check  ori_p  ori_fp
0  20  29   True     20      29
1  19  30  False     19      30
2  18  31   True     18      31

('681916 083118 x', '16E650')
len(point_table) = 4
[7 7 7 7]

    p fp check  ori_p  ori_fp
0  12  5  True     12       5
1  13  4  True     13       4
2  14  3  True     14       3
3  15  2  True     15       2

('681916 083118 x', '16E651')
len(point_table) = 2
[2 1]

   p fp  check  ori_p  ori_fp
0  8  9   True      8       9
1  9  8  False      9       8

('681916 083118 x', '16E652'

('682363 091018 x', '16E152')
len(point_table) = 3
[5 5 5]

   p  fp check  ori_p  ori_fp
0  6  11  True      6      11
1  7  10  True      7      10
2  8   9  True      8       9

('682363 091018 x', '16E153')
len(point_table) = 2
[1 2]
    p  fp  check  ori_p  ori_fp
0  19  30  False     19      30
1  18  31   True     18      31

('682363 091018 x', '16E155')
len(point_table) = 3
[2 2 2]

    p fp  check  ori_p  ori_fp
0  14  3   True     14       3
1  15  2  False     15       2
2  15  2   True     16       1

('683518 012218 x', '16F0AE')
len(point_table) = 4
[6 6 3 6]

   p  fp  check  ori_p  ori_fp
0  2  15   True      2      15
1  3  14   True      3      14
2  4  13  False      4      13
3  5  12   True      5      12

('683518 012218 x', '16F0AF')
len(point_table) = 2
[2 1]
    p  fp  check  ori_p  ori_fp
0  29  20   True     29      20
1  28  21  False     28      21

('683518 012218 x', '16F0B1')
len(point_table) = 1
[1]
   p  fp check  ori_p  ori_fp
0  7  10  True      7  

In [34]:
out = pd.concat([out_df,ori_df[['ori_p','ori_fp']]],axis=1)
out

,folder,image,label,flip_label,p,fp,check,point,ori_p,ori_fp
0,585387_2 091119 x,4C085,30,19,30,19,True,5,30,19
1,585387_2 091119 x,4C085,29,20,29,20,True,5,29,20
2,585387_2 091119 x,4C085,28,21,28,21,True,5,28,21
3,585387_2 091119 x,4C086,10,7,10,7,True,5,10,7
4,585387_2 091119 x,4C086,11,6,11,6,True,5,11,6
...,...,...,...,...,...,...,...,...,...,...
995,684988 92118 x,16DEE8,21,28,21,29,False,999,21,29
996,684988 92118 x,16DEE9,2,15,2,15,True,1,2,15
997,684988 92118 x,16DEE9,5,12,3,14,False,0,4,12
998,684988 92118 x,16DEEA,28,21,28,21,True,3,28,21


In [35]:
for radio_idx, radio_row in out_df.iterrows():
    if radio_row['check'] == True:
        out_df.loc[radio_idx,'check'] = 1
out_df = out_df.reindex(columns=['folder','image','label','p','flip_label','fp','check','point'])
out_df = out_df.set_index(keys = ['folder','image'])
out_df.to_csv('0831.csv')
out_df

label   p flip_label  fp  check  point
folder            image                                        
585387_2 091119 x 4C085     30  30         19  19      1      5
                  4C085     29  29         20  20      1      5
                  4C085     28  28         21  21      1      5
                  4C086     10  10          7   7      1      5
                  4C086     11  11          6   6      1      5
...                        ...  ..        ...  ..    ...    ...
684988 92118 x    16DEE8    21  21         28  29  False    999
                  16DEE9     2   2         15  15      1      1
                  16DEE9     5   3         12  14  False      0
                  16DEEA    28  28         21  21      1      3
                  16DEEA    27  27         22  22      1      3

[1000 rows x 6 columns]

In [36]:
ori_df = ori_df.set_index(keys = ['folder','image'])
ori_df

ori_p ori_fp
folder            image              
585387_2 091119 x 4C085     30     19
                  4C085     29     20
                  4C085     28     21
                  4C086     10      7
                  4C086     11      6
...                        ...    ...
684988 92118 x    16DEE8    21     29
                  16DEE9     2     15
                  16DEE9     4     12
                  16DEEA    28     21
                  16DEEA    27     22

[1000 rows x 2 columns]

In [37]:
out = out.reindex(columns=['folder','image','label','p','flip_label','fp','check','ori_p','ori_fp','point'])
out = out.set_index(keys = ['folder','image'])
out.to_csv('0831_ori_pred.csv')
out

label   p flip_label  fp  check ori_p ori_fp  point
folder            image                                                     
585387_2 091119 x 4C085     30  30         19  19   True    30     19      5
                  4C085     29  29         20  20   True    29     20      5
                  4C085     28  28         21  21   True    28     21      5
                  4C086     10  10          7   7   True    10      7      5
                  4C086     11  11          6   6   True    11      6      5
...                        ...  ..        ...  ..    ...   ...    ...    ...
684988 92118 x    16DEE8    21  21         28  29  False    21     29    999
                  16DEE9     2   2         15  15   True     2     15      1
                  16DEE9     5   3         12  14  False     4     12      0
                  16DEEA    28  28         21  21   True    28     21      3
                  16DEEA    27  27         22  22   True    27     22      3

[1000 rows x 8 columns]

In [38]:
acc_p = 0
acc_fp = 0
for idx, row in out.iterrows():
    if row['ori_p'] == row['label']:
        acc_p += 1
    if row['ori_fp'] == row['flip_label']:
        acc_fp += 1
acc = acc_p + acc_fp
print(acc_p/len(out_df), acc_fp/len(out_df),acc / (2*len(out_df)))

0.807 0.807 0.807


In [39]:
acc_p = 0
acc_fp = 0
for idx, row in out.iterrows():
    if row['p'] == row['label']:
        acc_p += 1
    if row['fp'] == row['flip_label']:
        acc_fp += 1
acc = acc_p + acc_fp
print(acc_p/len(out_df), acc_fp/len(out_df),acc / (2*len(out_df)))

0.881 0.885 0.883


In [23]:
x = np.concatenate((ori_df['ori_p'].values.astype(int),ori_df['ori_fp'].values.astype(int)))
y = np.concatenate((out_df['label'].values.astype(int),out_df['flip_label'].values.astype(int)))
plot_confusion_matrix(y, x,'ori_concate_prediction')